# [Shor's algorithm code from IBM's Qiskit used](https://qiskit.org/textbook/ch-algorithms/shor.html)
## Their docs are excellent, please take advantage of them!
## Code adapted for CirQ

In [1]:
import unittest

import cirq
from cirq.ops import H, X, I
import random
import matplotlib.pyplot as plt
from math import gcd
import numpy as np
from numpy.random import randint

import hypothesis.strategies as st
from hypothesis import given, settings

from fractions import Fraction
from math import gcd # greatest common divisor

In [2]:
# Specify variables
n_count = 8  # number of counting qubits
a = 7
N = 15

In [3]:
class aMod15Gate(cirq.Gate):
    def __init__(self, a, power):
        super(aMod15Gate, self)
        self.a = a
        self.power = power

    def _num_qubits_(self):
        return 4

    def _decompose_(self, qubits):
        q0, q1, q2, q3 = qubits
        if self.a not in [2,7,8,11,13]:
            raise ValueError("'a' must be 2,7,8,11 or 13")
        for iteration in range(self.power):
            if self.a in [2,13]:
                yield cirq.SWAP(q0,q1)
                yield cirq.SWAP(q1,q2)
                yield cirq.SWAP(q2,q3)
            if self.a in [7,8]:
                yield cirq.SWAP(q2,q3)
                yield cirq.SWAP(q1,q2)
                yield cirq.SWAP(q0,q1)
            if self.a == 11:
                yield cirq.SWAP(q1,q3)
                yield cirq.SWAP(q0,q2)
            if self.a in [7,11,13]:
                yield cirq.X(q0)
                yield cirq.X(q1)
                yield cirq.X(q2)
                yield cirq.X(q3)

    def _circuit_diagram_info_(self, args):
        return "a mod 15" 

In [4]:
def qft_dagger_cirq(qc, qubits, n):
    for qubit in range(n//2):
        qc.append(cirq.SWAP(qubits[qubit], qubits[n-qubit-1]))
    for j in range(n):
        for m in range(j):
            qc.append((cirq.CZ**(-1/2**(j-m)))(qubits[m],qubits[j]))
        qc.append(cirq.H(qubits[j]))

In [5]:
def qpe_amod15(a):
    n_count = 8
    qubits = cirq.LineQubit.range(4+n_count)
    qc = cirq.Circuit()     
    for q in range(n_count):
        #print(q)
        qc.append(cirq.H(qubits[q]))     # Initialize counting qubits in state |+>
    qc.append(cirq.X(qubits[3+n_count])) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(aMod15Gate(a, 2**q).on(qubits[8],qubits[9],qubits[10],qubits[11]).controlled_by(qubits[q]))
    qft_dagger_cirq(qc, qubits[:n_count], n_count) # Do inverse-QF
    qc.append(cirq.measure(*qubits[:8], key='m'))
    # Simulate Results
    simulator = cirq.Simulator()
    results = simulator.run(qc , repetitions =1)
    print(type(results.measurements['m'][0]))
    print(np.array2string(results.measurements['m'][0], separator='')[1:-1][::-1])
    readings = np.array2string(results.measurements['m'][0], separator='')[1:-1][::-1]
    phase = int(readings,2)/(2**n_count)
    return phase

In [6]:
phase = qpe_amod15(a) # Phase = s/r
Fraction(phase).limit_denominator(15)

<class 'numpy.ndarray'>
11000000


Fraction(3, 4)

In [7]:
frac = Fraction(phase).limit_denominator(15)
s, r = frac.numerator, frac.denominator
print(r)

4


In [8]:
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

[3, 5]


In [9]:
a = 7
factor_found = False
attempt = 0
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    phase = qpe_amod15(a) # Phase = s/r
    frac = Fraction(phase).limit_denominator(N) # Denominator should (hopefully!) tell us r
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 15)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            if guess not in [1,N] and (N % guess) == 0: # Check to see if guess is a factor
                print("*** Non-trivial factor found: %i ***" % guess)
                factor_found = True


Attempt 1:
<class 'numpy.ndarray'>
00000000
Result: r = 1

Attempt 2:
<class 'numpy.ndarray'>
10000000
Result: r = 2
Guessed Factors: 3 and 1
*** Non-trivial factor found: 3 ***
